In [26]:
using LinearAlgebra
using SwitchOnSafety
using ControlSystems
using StaticArrays
using HybridSystems
using JuMP
using MosekTools
using Random
using LightGraphs
using SpecialFunctions
using Arpack

In [27]:
CJSR = .95

0.95

In [28]:
A1 = [-0.12264385232852724 -0.5132986766380205; -0.512239944583497 0.7657920763290312]     * CJSR
A2 = [-1.0025775249390196 0.35564172966144936; -0.01094819510863365 0.5106100376225976]    * CJSR
# A3 = [-0.35630095375724796 -1.7520760336939292; -0.057098017100732236 0.9262405094853716]  * CJSR
# A4 = [-0.8304488630107022 -0.5488013010939014; -0.30728137981798154 -0.005394926625104089] * CJSR
# A5 = [2.968207574740448 -2.2374945860620774; 2.1714600819887067 -1.2999866350779252]       * CJSR
# Σ = [A1, A2, A3, A4, A5]
Σ = [A1, A2]

cycle = LightAutomaton(2)
add_transition!(cycle, 1, 2, 1)
add_transition!(cycle, 2, 1, 2)
nothing

In [32]:
hs = discreteswitchedsystem(Σ, cycle)
include("../../../src/WhiteBox.jl")
# @show white_box_CJSR_upper_bound(hs, 2)

CJSR = ρ(A1*A2)^(1/2)

0.6198954485161069

In [38]:
include("../../../src/Bounds.jl")
include("../../../src/Trajectories.jl")
VERBOSE = false

N = 1000
N_step = 100
N_begin = 10
N_range = N_begin:N_step:N

# Known information
dim = 2
m = 2
V = 2

# Parameters
β = .99
β1 = (β + 1.) / 2.
β2 = β1

for l = 1:8
    p_l = 1 / m
    quantity = 1 / p_l

    if VERBOSE @show m end
    total_time = 0

    f = open("res_CQLF_$l.txt", "w")
    println("Simulations start for l = $l")
    for n = N_range
        u, v, x, y = generate_trajectories(Σ, cycle, n, l)
        y = reshape(y[:, l, :], size(x))

        total_time += @elapsed upper_bound = upper_bound_CQLF(x, y, β, l, quantity, quantity_max = quantity)     
        println(f, "$upper_bound")
    end
    close(f)

    if VERBOSE @show total_time end
end

Simulations start for l = 1


Simulations start for l = 2


Simulations start for l = 3


Simulations start for l = 4


Simulations start for l = 5


Simulations start for l = 6


Simulations start for l = 7


Simulations start for l = 8


In [39]:
using PyPlot
using DelimitedFiles

COLORS = ["b", "g", "r", "c", "m", "y", "orange", "darkviolet"]

figure(figsize=(8, 5))
axhline(CJSR, linestyle="--", color="k", alpha=.7, label="\$\\rho(G, \\Sigma)\$")
for l = 1:8
    data_CQLF = readdlm("res_CQLF_$l.txt")
    keep_CQLF = map(x -> x != -1, data_CQLF)
    keep_CQLF = reshape(keep_CQLF, size(N_range))
    plot(N_range[keep_CQLF], (data_CQLF)[keep_CQLF], "-", color=COLORS[l], label="\$l = $l\$")
end
legend()
title("CQLF upper bound")
# yscale("log")
xlabel("Number of observations \$N\$")
margins(x=0)
PyPlot.grid()
show()
savefig("trace_length_CQLF_ub.pdf")

sys:1: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
